<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/real_summary3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Beginners Guide to Text Generation using LSTMs**

https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch


In [45]:
!pip install sentence-transformers

In [52]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# 여기서부터 본론...

In [3]:
from sentence_transformers import SentenceTransformer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

In [4]:
document = """
주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.
주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 '군사작전'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 '패스트트랙'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.
이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 '공수처는 야당의 동의 없이는 절대 출범할 수 없는 것'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 '비토권'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.
주 원내대표는 "이해찬 전 민주당 대표가 얘기한 '민주당 20년 집권'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 '민주주의 없는 민주당'이 법위에 군림하는 '반민주'를 거리낌없이 획책하는 것"이라고 언급했다.
그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.
주 원내대표는 "문 대통령은 제게 '공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다'고 했는데, 그런 분이 청와대와 대통령 주변을 감시하는 특별감찰관은 취임 이후 지금까지 왜 임명하지 않았는가"라며 "공수처는 권력형 비리의 쓰레기 하치장, 종말 처리장이 될 것"이라고 비판했다.
문재인 정부를 향해 주 원내대표는 "문 대통령과 그 사도들은 법치가 미치지 않는 무오류의 화신이 될 것"이라며 "오류를 인정하지 않는 존재가 바로 신이며 그 아래에는 자신들의 지도자를 목숨바쳐 지킴으로서 정의를 실현하겠다는 추종자들로 넘쳐 난다. 공수처는 지도자의 신성을 인정하지 않는 세력을 정죄하는 수단으로 전락할 것"이라고 질타했다.
주 원내대표는 "저도 법조인이지만 대통령과 공수처장이 마음대로 검사들과 수사관들을 임명하는 이 끔찍한 사법기구가 어떤 일을 할지 두렵기만 하다"며 "공수처는 검찰과 경찰 위에 있는 사법기구로, 헌법과 법으로 독립성을 보장하는 검찰총장을 이렇게 핍박하는 정권이 공수처를 어떻게 운영할지 불을 보듯 뻔한 일"이라고 예측했다.
그러면서 주 원내대표는 "추미애 법무장관을 앞장 세워 윤석열 검찰의 권력 비리 수사를 저지하려다가 난관에 봉착하자 무슨 수를 써서라도 공수처를 출범시키려 한다. 공수처장 자리에는 추미애보다 더 한 막무가내 내 편을 앉힐 게 분명한 것"이라며 "문 정권의 파렴치와 오만함을 최전선에서 온 몸으로 겪어온 저로서는 민주당이 내일부터 국회에서 보일 행태가 환히 보인다. 180석의 민주당이 또 군사작전을 개시하면 그걸 누가 막겠는가"라고 성토했다.
주 원내대표는 "공수처법을 막을 힘이 우리 야당에게는 없다. 삭발하고 장외투쟁해 봐야 눈 하나 깜짝할 사람들이 아닌 것"이라며 "대란대치(大亂大治), 세상을 온통 혼돈 속으로 밀어넣고 그걸 권력 유지에 이용한다는 게 이 정권의 통치기술"이라고 규탄했다.
아울러 주 원내대표는 "권력은 바람, 국민은 풀이다. 바람이 불면 청보리 밭의 보리가 눕는다"며 "권력은 풀들이 다시는 일어서지 못하도록 풀을 짓밟지만 풀들은 다시 일어난다. 시인 김수영은 '바람보다 먼저 눕지만, 바람보다 먼저 일어나는' 민초의 힘을 노래했다"고 말했다.
마지막으로 주 원내대표는 "문재인 정권은 이제 곧 국회에서 광장에서 짓밟힌 풀들이 일어서서 아우성치는 모습을 지켜보게 될 것"이라며 "대란대치를 끝장내려는 국민적 저항에 직면할 것"이라고 거듭 강조했다.
"""

In [47]:
c = embedder.encode([document])
c

array([[ 0.53171986, -0.08288396,  0.4629747 , ..., -0.54250145,
        -0.10200596, -0.05710737]], dtype=float32)

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([document])
total_words = len(tokenizer.word_index) + 1

In [7]:
print(total_words)

395


In [94]:
word_keys = []
word_values = []

for word,index in tokenizer.word_index.items():
    word_keys.append(index)
    word_values.append(word)

word_keys.append(len(word_values)+1)
word_values.append('[   ]')

for k in word_keys:
  print(k,word_values[k-1])

1 원내대표는
2 주
3 것
4 이라고
5 며
6 문재인
7 않는
8 문
9 이라며
10 이제
11 풀들이
12 수
13 대통령과
14 공수처는
15 될
16 있는
17 향해
18 보인다
19 짓밟힌
20 국민적
21 저항에
22 직면할
23 정권이
24 위해
25 공수처법을
26 제게
27 대통령은
28 '공수처는
29 없는
30 야당이
31 공수처장
32 사람들이
33 민주당
34 그러면서
35 공수처가
36 청와대와
37 왜
38 그
39 인정하지
40 이
41 공수처를
42 권력
43 게
44 정권의
45 민주당이
46 국회에서
47 그걸
48 권력은
49 먼저
50 주호영
51 국민의힘
52 22일
53 고위공직자범죄수사처
54 공수처
55 법
56 개정과
57 가덕도
58 신공항
59 건설
60 등을
61 밀어붙이고
62 정권과
63 더불어민주당을
64 끝이
65 아우성
66 치는
67 경고했다
68 이날
69 자신의
70 페이스북에
71 공수처법
72 개정을
73 위한
74 '군사작전'에
75 돌입하겠다고
76 엄포를
77 놓고
78 있다
79 정의당을
80 끌어들이기
81 꼼수
82 선거법에
83 묶어
84 '패스트트랙'이라는
85 불법·탈법으로
86 만들어낸
87 시행도
88 해보지
89 않고
90 고치려
91 하는
92 지적했다
93 이어
94 야당
95 원내대표인
96 사람
97 좋아보이는
98 표정으로
99 야당의
100 동의
101 없이는
102 절대
103 출범할
104 것'이라고
105 얘기했고
106 유엔
107 안보리
108 상임이사국처럼
109 임명에
110 '비토권'을
111 행사할
112 있는데
113 무얼
114 걱정하느냐고
115 여당
116 우리를
117 속였다
118 거짓말이라는
119 비난을
120 개의치
121 사람들
122 꼬집었다
123 이해찬
124 전
125 대표가
126 얘기한
127 '민주당
128 20년
129 집권'의
130 토대가
131 올해
132 안에
133 완성된다
134 탈원전과
135 동남

In [53]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input,
                                     Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten)
import matplotlib.pyplot as plt

In [95]:
#-*- coding: utf-8 -*-

keys=tf.constant(word_keys,tf.int32)
values=tf.constant(word_values, tf.string)
# build a lookup table
word_table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(keys,values),
    default_value=tf.constant(' '),
    name="class_weight"
)


# build a lookup table
#word_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(tf.constant(word_keys), tf.constant(word_values)),default_value=-1)


In [11]:
#-*- coding: utf-8 -*-

a = tf.constant("한글")
print(a)


tf.Tensor(b'\xed\x95\x9c\xea\xb8\x80', shape=(), dtype=string)


In [133]:
word_table.lookup(tf.constant(0)).numpy().decode('utf-8')

' '

In [68]:
import sys

def to_text(w):
    #r_value = None
    texts = []
    try:
        #codes = []
        
        #embeddings = []

        for z in w:
            text = ""
            #code = []
            for v in z:
                try:
                    #print(v)
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    #if type=='code':
                    #    code.append(key.numpy())
                    #else:
                    #    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                except Exception as ex:
                    print(ex,sys.exc_info())
                    #code.append(0)
                    text += '[   ] '                  
            #print(generated_text)
            #if type == 'code':
            #    codes.append(code)
            #else:
            #    texts.append(text)
            texts.append(text)

    except Exception as ex:
        print(ex,sys.exc_info())
        #codes = np.random([w.shape[0],30])
        #texts = '----------------------------------------------------'
    '''
    if type == 'code':
        r_value = tf.constant(codes,dtype=tf.int32)
    elif type == 'embedding':
        r_value = tf.constant(embedder.encode(texts),dtype=tf.float32)
    else:
        r_value = tf.constant(texts,dtype=tf.string)
    '''
    return tf.constant(texts,dtype=tf.string)



In [99]:
w = tf.random.normal([2,30,total_words])
e = to_text(w)
print(e)
for t in e:
  print(t.numpy().decode('utf-8'))

tf.Tensor(
[b"\xec\x84\xa0\xea\xb1\xb0\xeb\xb2\x95\xec\x97\x90 \xec\xb6\x9c\xeb\xb2\x94\xed\x95\xa0 \xec\x9a\xb0\xeb\xa6\xac\xeb\xa5\xbc \xec\x9c\x84\xed\x95\xb4 \xeb\xaa\xbb\xed\x95\x98\xeb\x8f\x84\xeb\xa1\x9d '\xeb\xb0\x94\xeb\x9e\x8c\xeb\xb3\xb4\xeb\x8b\xa4 \xec\x9d\xb4\xed\x9b\x84 \xeb\xb6\x80\xec\xa0\x95\xed\x95\x98\xeb\x8a\x94 \xea\xb3\xb5\xec\x88\x98\xec\xb2\x98\xeb\x8a\x94 \xec\x83\x81\xec\x9e\x84\xec\x9d\xb4\xec\x82\xac\xea\xb5\xad\xec\xb2\x98\xeb\x9f\xbc \xea\xb2\x80\xec\xb0\xb0\xec\x9d\x98 \xec\x93\xb0\xeb\xa0\x88\xea\xb8\xb0 \xea\xb0\x80\xeb\x8d\x95\xeb\x8f\x84 \xea\xb0\x81\xec\xa2\x85 \xeb\xaf\xbc\xec\xa3\xbc\xeb\x8b\xb9\xec\x9d\xb4 \xeb\x82\x9c\xeb\x8b\xa4 \xea\xb2\x83\xec\x9d\xb4\xeb\x9d\xbc\xea\xb3\xa0 \xea\xb7\xb8\xeb\x9f\xb0 \xea\xb2\x83\xec\x9d\xb4\xeb\x8b\xa4 '\xea\xb5\xb0\xec\x82\xac\xec\x9e\x91\xec\xa0\x84'\xec\x97\x90 \xea\xb0\x9c\xec\x8b\x9c\xed\x95\x98\xeb\xa9\xb4 \xec\x9d\xb4\xec\xa0\x9c \xec\xa0\x95\xec\x9d\x98\xeb\x8b\xb9\xec\x9d\x84 \xea\xb0\x80\xec\xa0\xb8

In [100]:
def to_embedding(w):
    texts = []
    try:
        for z in w:
            text = ""
            for v in z:
                try:
                    key = tf.argmax(v,output_type=tf.int32) #tf.constant(,dtype=tf.int32)
                    text += word_table.lookup(key).numpy().decode('utf-8') + ' '
                except Exception as ex:
                    print(ex,sys.exc_info())
                    text += '[   ] '                  
            texts.append(text)
    except Exception as ex:
        print(ex,sys.exc_info())

    return tf.constant(embedder.encode(texts),dtype=tf.float32)

In [101]:
e = to_embedding(w)
print(e)
for t in e:
  print(t)

tf.Tensor(
[[ 0.24092221 -0.44310427  0.6487462  ... -0.43285555  0.02594399
   0.15938537]
 [ 0.21037112  0.1266987   0.36156273 ... -0.1705244   0.0052964
   0.03188725]], shape=(2, 1024), dtype=float32)
tf.Tensor(
[ 0.24092221 -0.44310427  0.6487462  ... -0.43285555  0.02594399
  0.15938537], shape=(1024,), dtype=float32)
tf.Tensor(
[ 0.21037112  0.1266987   0.36156273 ... -0.1705244   0.0052964
  0.03188725], shape=(1024,), dtype=float32)


In [102]:
from keras import backend as K
from keras.layers import Layer

#tf.executing_eagerly()

class post_processing(Layer):

    def __init__(self, output_dim, encoder_func,Tout=tf.float32, **kwargs):
        self.output_dim = output_dim
        self.encoder = encoder_func
        self.Tout = Tout
        super(post_processing, self).__init__(**kwargs)
    '''
    def build(self, input_shape):
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(sentence_encoder, self).build(input_shape)  # 끝에서 꼭 이 함수를 호출하십시오
    '''
    def call(self, x):
        return tf.py_function(self.encoder,[x],Tout=self.Tout)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [103]:
e = post_processing(1024,to_embedding,Tout=tf.float32,trainable=True)(w)

for c in e:
  print(c)

tf.Tensor(
[ 0.24092221 -0.44310427  0.6487462  ... -0.43285555  0.02594399
  0.15938537], shape=(1024,), dtype=float32)
tf.Tensor(
[ 0.21037112  0.1266987   0.36156273 ... -0.1705244   0.0052964
  0.03188725], shape=(1024,), dtype=float32)


In [104]:
e = post_processing(1,to_text,Tout=tf.string,trainable=True)(w)

for c in e:
  print(c.numpy().decode('utf-8'))

선거법에 출범할 우리를 위해 못하도록 '바람보다 이후 부정하는 공수처는 상임이사국처럼 검찰의 쓰레기 가덕도 각종 민주당이 난다 것이라고 그런 것이다 '군사작전'에 개시하면 이제 정의당을 가져가 것이다 분이 해보지 허물어뜨렸고 그만 대표가 
며 사건을 거리낌없이 사업이니 야당이 온 언급했다 얻기 동의 야당이 이권을 가덕도 처리장이 있다 건설 취임 운영할지 앞장 세워 먼저 꼼수 가담할 눕는다 불을 이용한다는 돌입하겠다고 완성된다 있다 거듭 '공수처는 


In [118]:
def make_generator_model(max_length,total_words):
    input = Input(shape=(1000,), dtype='int32') 
    x1 = Dense(1024, use_bias=False)(input)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    #x1 = Dense(max_length*total_words*2, use_bias=False)(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = LeakyReLU()(x1)
    
    x1 = Dense(max_length*total_words, use_bias=False)(x1)
    x1 = BatchNormalization()(x1)
    x1 = Softmax()(x1)        
    x1 = Reshape((max_length, total_words))(x1)
    t = post_processing(1,to_text,Tout=tf.string)(x1)
    e = post_processing(1024,to_embedding,Tout=tf.float32)(x1)

    model = Model(input,[t,e])
    
    '''
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(1000,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    assert model.output_shape == (None,256) 

    model.add(Dense(256*2, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    assert model.output_shape == (None,256*2) 
    
    model.add(Dense(total_words, use_bias=False))
    model.add(BatchNormalization())
    model.add(Softmax())

    assert model.output_shape == (None,total_words)
    '''
    model.summary()
    return model

generator = make_generator_model(30,total_words)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
dense_34 (Dense)                (None, 1024)         1024000     input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_22 (BatchNo (None, 1024)         4096        dense_34[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_22 (LeakyReLU)      (None, 1024)         0           batch_normalization_22[0][0]     
____________________________________________________________________________________________

In [119]:
# Create a random noise and generate a sample
noise = tf.random.normal([3,1000])
texts,embeddings = generator(noise, training=False)

for txt in texts:
    print(txt.numpy().decode('utf-8'))
    print('')

for cd in embeddings:
    print(cd.numpy())
    print('')

윤건영 주변을 놓고 밀어넣고 꼼수 윤석열 비판했다 인정하지 내 임명하는 권력 최전선에서 건설 게 독립성을 법조인이지만 권력이 목숨바쳐 것'이라고 공직자들을 원내대표는 그런 허물어뜨렸고 없다'고 대통령 끝이 걱정하느냐고 사건이 정권은 뻔한 

사람들이 최전선에서 개의치 끌어들이기 자신의 감시하는 하는 거짓말이라는 그걸 게 이해찬 '패스트트랙'이라는 사건이 완성된다 시인 두렵기만 김수영은 대란대치 출범시키려 대통령 신성을 추종자들로 大亂大治 않는 막겠는가 온통 사건을 혼돈 요직에 추종자들로 

써서라도 22일 추종자들로 표정으로 공수처는 걱정하느냐고 표정으로 수단으로 원내대표인 처리장이 개의치 그걸 신중함은 얘기한 수사를 온 검찰의 막겠는가 22일 임명에 공수처장이 세워 것'이라고 우리를 앉힐 규탄했다 눕지만 힘이 있다 경고했다 

[ 0.49488762 -0.47056934  0.39353213 ... -0.53949434  0.02929593
 -0.19521973]

[ 0.5782575  -0.1370514   0.9492064  ...  0.08604667 -0.00169895
 -0.4184904 ]

[ 0.49945843  0.22054337  0.33399284 ... -0.40973312 -0.2892311
 -0.17764106]



In [120]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(1024,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    #model.add(Reshape((256,)))
    assert model.output_shape == (None,256) # Note: None is the batch size

    model.add(Dense(64, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,64) # Note: None is the batch size

    model.add(Dense(32, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,32) # Note: None is the batch size

    model.add(Dense(1))
    #model.add(Softmax())    
    model.summary()
    return model

discriminator = make_discriminator_model()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 256)               262144    
_________________________________________________________________
batch_normalization_24 (Batc (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 64)                16384     
_________________________________________________________________
batch_normalization_25 (Batc (None, 64)                256       
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 64)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 32)               

In [121]:
#generated_encode = embedder.encode([generated_sum])
decision = discriminator(embeddings)
print (decision)

tf.Tensor(
[[-0.23204869]
 [-0.48966113]
 [-0.3612647 ]], shape=(3, 1), dtype=float32)


In [123]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [124]:
BATCH_SIZE = 32
EPOCHS = 60
noise_dim = 1000
seed = tf.random.normal([1, noise_dim])

In [125]:
@tf.function
def train_step(real_embedding):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      texts,embeddings = generator(noise, training=True)
      real_output = discriminator(real_embedding, training=True)
      fake_output = discriminator(embeddings, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    
    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [126]:
doc_emb = embedder.encode([document])[0]
print(doc_emb.shape)
dataset = []
for i in range(10):
  batch_set = []
  for j in range(BATCH_SIZE):
    batch_set.append(doc_emb)

  batch_set = np.asarray(batch_set)
  dataset.append(batch_set)

(1024,)


In [127]:
import time
from IPython import display # A command shell for interactive computing in Python.

def train(dataset, epochs):
  # A. For each epoch, do the following:
  for epoch in range(epochs):
    start = time.time()
    # 1 - For each batch of the epoch, 
    for image_batch in dataset:
      # 1.a - run the custom "train_step" function
      # we just declared above
      #print(image_batch.shape)
      train_step(image_batch)

    # 4 - Print out the completed epoch no. and the time spent
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [128]:
train(dataset, EPOCHS)

UnknownError: ignored